<b> <font size=5> Daymet Data API call </b> </font>

This iPython Notebook gets the annual average meteorological parameters from Oakridge National Lab's  Daymet dataset. Daymet contains meteorological parameters like Radiation, Temperature, Precipitation, and Pressure on a 1 km by 1 km grid, on a daily basis. The daily measurements were obtained using the Daymet API, and averaged over the timeperiod between June 2015 - May 2016.

In [1]:
#Import the daymetpy package
import daymetpy

#Import basic python packages for data analysis and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap
import math

from matplotlib import cm

import matplotlib.lines as mlines

import netCDF4 as nc

from netCDF4 import Dataset


import seaborn as sns

from multiprocessing.pool import ThreadPool


import time


%matplotlib inline

# <b> <font size = 5> Load Monitoring Dataset </b> </font>

## <b> <font size = 4> Get location of measurements </b> </font>

In [13]:
df_measured = pd.read_csv('EDF_Data.csv', header = 1)

In [14]:
# Extracting only the latitude and longitude
df_measured = df_measured[['Latitude','Longitude']]

In [15]:
df_measured.head()

,Latitude,Longitude
0,37.806781,-122.322594
1,37.806150,-122.322310
2,37.806420,-122.322301
3,37.805880,-122.322299
4,37.806689,-122.322267


## <b> <font size = 4> Get location of measurements </b> </font>

In [2]:
df_grid = pd.read_csv("Data/grid_points_oakland.csv")

In [3]:
df_grid.rename(columns = {'Lat': 'Latitude', 'Long':"Longitude"}, inplace=True)

In [4]:
df_grid.columns

Index(['Unnamed: 0', 'Latitude', 'Longitude', 'geometry', 'is_land'], dtype='object')

In [5]:
df_grid.drop(columns = ['Unnamed: 0', 'geometry', 'is_land'], inplace=True)

In [6]:
df_grid.shape

(2411, 2)

# <b> <font size = 5> Daymet API Call </b> </font>

## <b> <font size = 4> Function to make API call </b> </font>

<b> Create a function to make API call to daymet using daymetpy package. API call takes in arguments like latitude, longitude, start year, end year and output type as_dataframe </b>

In [11]:
def make_api_call(lat, lon, start_year, end_year):
    try:
        return daymetpy.daymet_timeseries(lon=lon, lat=lat, start_year=start_year, end_year=end_year, as_dataframe=True)
    except:
        return None

## <b> <font size = 4> Function to calculate average parameters within a defined time period </b> </font>

<b> Next, define a function that takes the dataframe, and calculates average parameters between Jun 2015 - May 2016 </b>

In [12]:
def daymet_avg(df1, year1, year2):
    df1.reset_index(inplace=True) #Reset index such that we have a column as YYYY-MM-DD
    df1.rename(columns={'index':'Datestamp'}, inplace=True) #Rename index as Datestamp
    start_date = str(year1) + '-06-01' #Set start date
    end_date = str(year2) + '-05-31'   #Set end data
    df1 = df1[(df1['Datestamp']>= start_date) & (df1['Datestamp'] <= end_date)] # Filter for days only between June 2015 to May 2016
    df1.drop(columns=['swe'], inplace = True) #Drop snow water equivalent since its always zero in Oakland
    df1_avg = np.mean(df1) #Calculate the average value of all measurements in a given location
    return (df1_avg['dayl'],df1_avg['prcp'],df1_avg['srad'], df1_avg['tmax'] ,df1_avg['tmin'] ,df1_avg['vp'])

daily_met_average_data = []
start_year = 2015
end_year = 2016
for row in df_measured.head(10.iterrows():
    lon, lat = (row[1]['Longitude'], row[1]['Latitude'])
    daymet_raw_df = make_api_call(lat, lon, start_year, end_year)
    daily_met_average_data.append(daymet_avg(daymet_raw_df, start_year, end_year)) ## create a list with daymet parameters for each location

compute_daily_met_average_df = pd.DataFrame(daily_met_average_data) #convert list to a dataframe

##### <font size = 3>  Add latitude and longitude daymet_average dataframe </font>
daymet_average = daymet_average.join(df_measured)

##### <font size = 3>  Write to a csv </font>
daymet_average.to_csv("Data/daymet_avg_15_16.csv")

## <b> <font size = 4> Make the API call; Use threadpool for parallelizing API calls. </b> </font>

pool = ThreadPool(processes=20)
start_year = 2015
end_year = 2016


async_result = [] #Create an empty list which stores the return object of the function call. 
                  #Results of the function call are later obtained using get()

    
# Run through the first 10,000 rows in the dataframe    
for row in df_measured.head(10000).iterrows():
    lon, lat = (row[1]['Longitude'], row[1]['Latitude'])
    thread_result = pool.apply_async(make_api_call, (lat, lon, start_year, end_year))
    async_result.append(thread_result)

#Note that the function name and arguments are passed differently to the apply_async function. 
#async_result now stores the returned object of the make_api_call

    
daymet_average_data_all = [] #This list stores the result of the api call for all the latitudes and longitudes

for res in async_result: #loop through each async_result object
    df_daymet_api = res.get() #using the .get() function to get the result
    if df_daymet_api is not None:
        # get the return value from function.
        daymet_api_avg_data_lat_lon = daymet_avg(df_daymet_api, start_year, end_year)  
        ## create a list with daymet parameters for each location
        daymet_average_data_all.append(daymet_api_avg_data_lat_lon) 
    else:
        daymet_average_data_all.append(None) 

#Identify the rows that did not return any value from the API call
resnone = [i for i, val in enumerate(daymet_avg_data_bottom) if val == None] 

resnone

## <b> <font size = 4> Make the API call again for rows that did not return any value with the previous API call.  <b> </font> 

**Append the results of this to daymet_average_data_all created above.**

daymet_avg_data_top  = daymet_average_data_all

In [28]:
def process_nones(df_measured, daymet_average_data_all):
    indexes_with_none = [i for i, val in enumerate(daymet_average_data_all) if val == None]
    if not indexes_with_none:
        return

    pool = ThreadPool(processes=1)
    start_year = 2015
    end_year = 2016

    async_result = [] #Create an empty list which stores the 
    #return object of the function call. Results of the function call are later obtained using get()

    for idx in indexes_with_none:
        lon, lat = (df_measured.iloc[idx]['Longitude'], df_measured.iloc[idx]['Latitude'])
        thread_result = pool.apply_async(make_api_call, (lat, lon, start_year, end_year))
        async_result.append(thread_result)

    #Note that the function name and arguments are passed differently to the apply_async function. 
    #async_result now stores the returned object of the make_api_call

    for i, res in enumerate(async_result): #loop through each async_result object
        df_daymet_api = res.get() #using the .get() function to get the result
        print(df_daymet_api)
        daymet_api_avg_data_lat_lon = daymet_avg(df_daymet_api, start_year, end_year)  
        idx = indexes_with_none[i]
        daymet_average_data_all[idx] = daymet_api_avg_data_lat_lon 
#        if df_daymet_api is not None:
            # get the return value from function.
            
            ## create a list with daymet parameters for each location


process_nones(df_measured, daymet_avg_data_top)

In [29]:
def get_daymet_average_data_for_df(df_measured):
    pool = ThreadPool(processes=20)
    start_year = 2015
    end_year = 2016


    async_result = [] #Create an empty list which stores the 
    #return object of the function call. Results of the function call are later obtained using get()

    for row in df_measured.iterrows():
        lon, lat = (row[1]['Longitude'], row[1]['Latitude'])
        thread_result = pool.apply_async(make_api_call, (lat, lon, start_year, end_year))
        async_result.append(thread_result)
        print(lon,lat)

    #Note that the function name and arguments are passed differently to the apply_async function. 
    #async_result now stores the returned object of the make_api_call


    daymet_average_data_all = [] #This list stores the result of the api call for all the latitudes and longitudes

    for res in async_result: #loop through each async_result object
        df_daymet_api = res.get() #using the .get() function to get the result
        if df_daymet_api is not None:
            # get the return value from function.
            daymet_api_avg_data_lat_lon = daymet_avg(df_daymet_api, start_year, end_year)  
            ## create a list with daymet parameters for each location
            daymet_average_data_all.append(daymet_api_avg_data_lat_lon) 
        else:
            daymet_average_data_all.append(None)
            
    process_nones(df_measured, daymet_average_data_all)
    return daymet_average_data_all

In [52]:
daymet_avg_data_top = get_daymet_average_data_for_df(df_measured.head(10000))
daymet_avg_data_bottom = get_daymet_average_data_for_df(df_measured.tail(11488))



-122.1665205 37.76192083
-122.16650659999999 37.76251131
-122.1665065 37.74882955
-122.1665043 37.75322826
-122.1664893 37.76467032
-122.1664791 37.75072567
-122.1664699 37.75442551
-122.16646940000001 37.77162127
-122.16646770000001 37.74200214
-122.16646680000001 37.77161883
-122.1664612 37.75785653
-122.16645940000001 37.74626814
-122.1664591 37.75771618
-122.1664538 37.77476486
-122.16645049999998 37.74484093
-122.16643540000001 37.74815795
-122.16643390000002 37.74205518
-122.16643219999999 37.75687944
-122.16642879999999 37.75262901
-122.16642649999999 37.76112478
-122.1664129 37.77411136
-122.1664118 37.74350871
-122.16640990000002 37.76445434
-122.16639740000002 37.74671429
-122.16639070000001 37.76687676
-122.1663696 37.75466734
-122.1663487 37.74132128
-122.16633829999999 37.76655651
-122.1663335 37.74859667
-122.16633229999998 37.7417971
-122.1663314 37.76227946
-122.16632299999999 37.76966661
-122.1663206 37.75215656
-122.1663195 37.74447521
-122.16631159999999 37.75010235


-122.1598519 37.74771533
-122.15984830000001 37.7429081
-122.1598478 37.75154101
-122.1598436 37.74454779
-122.1598375 37.74949606
-122.1598254 37.761771100000004
-122.1598192 37.75587622
-122.15980320000001 37.76403448
-122.15979890000001 37.76406478
-122.1597986 37.75448894
-122.15977290000001 37.74234119
-122.15977269999999 37.7590391
-122.15976540000001 37.75725485
-122.15975290000002 37.74537057
-122.15975149999998 37.75128165
-122.15974979999999 37.77009513
-122.1597422 37.76920601
-122.1597387 37.76814632
-122.1597256 37.74383481
-122.1597236 37.74628451
-122.15971840000002 37.76680194
-122.15971409999999 37.74942787
-122.15971029999999 37.76429461
-122.1597041 37.74574062
-122.15970209999999 37.76319815
-122.15970060000001 37.75423
-122.15969579999998 37.74747502
-122.1596773 37.74208172
-122.15967309999999 37.76579832
-122.159673 37.74925931
-122.1596686 37.76653118
-122.1596658 37.74211255
-122.1596591 37.74780851
-122.15964369999999 37.75715905
-122.1596219 37.77461423
-122.

-122.15251640000001 37.76469364
-122.1525128 37.74904114
-122.15250549999999 37.75376297
-122.15250230000001 37.74563473
-122.1524976 37.76718955
-122.15249399999999 37.75134981
-122.15248970000002 37.75463313
-122.1524739 37.756105299999994
-122.1524656 37.75379397
-122.15243390000002 37.74598154
-122.1524207 37.76479849
-122.1524174 37.74511602
-122.1524057 37.75836993
-122.15240390000001 37.74878497
-122.15239050000001 37.76444241
-122.1523877 37.75758937
-122.15237520000001 37.75109642
-122.15236019999999 37.74867881
-122.1523601 37.76694219
-122.1523498 37.75400586
-122.1523481 37.743720700000004
-122.1523418 37.74432614
-122.15230929999998 37.74781645
-122.15229950000001 37.75440885
-122.15229479999999 37.76454727
-122.1522841 37.7558959
-122.1522753 37.75588567
-122.1522743 37.74583551
-122.15226809999999 37.74703741
-122.1522646 37.76419118
-122.15225690000001 37.75084289
-122.1522479 37.76097678
-122.15224029999999 37.753989000000004
-122.15223909999999 37.75450296
-122.152220

-122.2744121 37.79908747
-122.27440459999998 37.79909949
-122.27439950000002 37.79885843
-122.2743797 37.80168652
-122.27436709999999 37.79613331
-122.2743574 37.80172236
-122.2743561 37.79446522
-122.2743416 37.80041614
-122.27434140000001 37.79874774
-122.27432820000001 37.803189399999994
-122.27432560000001 37.79785019
-122.2743254 37.79686419
-122.2743217 37.80884765
-122.2743044 37.79943668
-122.27429609999999 37.80721266
-122.27429380000001 37.80417424
-122.27428870000001 37.80296556
-122.2742751 37.8012448
-122.27426969999999 37.80657487
-122.27426270000001 37.80894222
-122.2742557 37.79523165
-122.27425420000002 37.79462769
-122.274253 37.79934166
-122.27422849999999 37.80192882
-122.2742233 37.79773807
-122.2742153 37.79693289
-122.2741946 37.80378708
-122.2741794 37.7971085
-122.27417630000001 37.79949504
-122.27417150000001 37.79524615
-122.2741642 37.80878112
-122.27416170000001 37.80961839
-122.2741608 37.799489
-122.27415970000001 37.79784336
-122.2741587 37.80548579
-122

-122.26928190000001 37.81915345
-122.2692801 37.79318419
-122.2692716 37.79667762
-122.26925619999999 37.81358396
-122.2692554 37.80265517
-122.2692517 37.80184624
-122.2692436 37.81948147
-122.26923359999999 37.81650759
-122.26923329999998 37.81632979
-122.26923219999999 37.81947208
-122.26922759999998 37.79562005
-122.2692264 37.80505713
-122.2692262 37.811019
-122.26922590000001 37.79798519
-122.2692216 37.819544799999996
-122.26921809999999 37.82037079
-122.26921499999999 37.79754742
-122.2692132 37.81879162
-122.26920790000001 37.81109115
-122.2692033 37.81869098
-122.26919409999999 37.81521768
-122.26919380000001 37.79228469
-122.26919070000001 37.8193752
-122.26918570000001 37.80040194
-122.26917830000001 37.80267598
-122.2691777 37.79583284
-122.2691655 37.80940654
-122.26915890000002 37.81128406
-122.26915559999999 37.80008842
-122.2691517 37.80782922
-122.26914690000001 37.81980861
-122.2691467 37.80473484
-122.26914440000002 37.8231688
-122.2691424 37.82063441
-122.269132599

...

-122.22336750000001 37.77162628
-122.22336489999999 37.79965386
-122.22334679999999 37.79964666
-122.2232389 37.7712652
-122.22317790000001 37.7716684
-122.22310929999999 37.79980679
-122.2230942 37.77146491
-122.2230351 37.79958585
-122.2229851 37.77108497
-122.2229188 37.79977612
-122.2228308 37.77129364
-122.22278 37.79973712
-122.22274250000001 37.770895200000005
-122.2227053 37.79951784
-122.2225841 37.79972537
-122.2225739 37.77111609
-122.22251000000001 37.77069765
-122.22245079999999 37.79966744
-122.22237549999998 37.79944983
-122.2223226 37.77093364
-122.2222896 37.77049168
-122.2222494 37.79967461
-122.22212160000001 37.79959776
-122.2220842 37.77074075
-122.22207990000001 37.77027861
-122.2220457 37.79938182
-122.2219733 37.77016919
-122.22191470000001 37.79962386
-122.22185630000001 37.77053979
-122.22179229999999 37.79952808
-122.2217797 37.76994673
-122.22171589999999 37.7993138
-122.2216466 37.77032693
-122.22159450000001 37.76972001
-122.22158 37.7995731
-122.2214693 3

-122.1872044 37.7303328
-122.18710349999999 37.72988313
-122.1871031 37.72973649
-122.1870991 37.78410061
-122.1870128 37.78428623
-122.1870124 37.73010953
-122.1869097 37.72966085
-122.1868701 37.72953931
-122.18682040000002 37.72988625
-122.1867595 37.78408002
-122.1867159 37.72943858
-122.1866731 37.78426701
-122.18663719999999 37.72934213
-122.1866284 37.72966297
-122.18659479999998 37.72982874
-122.186522 37.72921632
-122.1864364 37.72943969
-122.186427 37.72959348
-122.18641940000002 37.78406387
-122.1863328 37.78425324
-122.1863281 37.72899406
-122.18626 37.72935789
-122.1862444 37.72921641
-122.18613429999999 37.72877181
-122.18610459999998 37.72911733
-122.186079 37.78405262
-122.1860524 37.72899313
-122.18599240000002 37.78424237
-122.18595149999999 37.72887582
-122.1859404 37.72854956
-122.18586040000001 37.72876985
-122.18574650000001 37.7283273
-122.1857386 37.78404138
-122.18566840000001 37.72854657
-122.1856615 37.72822983
-122.185652 37.7842315
-122.1854764 37.72832329


-122.15134450000001 37.74952521
-122.15132129999999 37.75013468
-122.1513199 37.74959633
-122.15131399999999 37.75011987
-122.15131219999999 37.75366265
-122.1512972 37.75344335
-122.15127530000001 37.7647552
-122.151252 37.69997537
-122.151252 37.76506496
-122.15124750000001 37.74926603
-122.15124740000002 37.75397686
-122.151222 37.75401059
-122.1511967 37.74934426
-122.15119059999999 37.74986787
-122.15115700000001 37.75368968
-122.1511486 37.69969393
-122.15114640000002 37.74900786
-122.1511212 37.75367372
-122.1510878 37.76452946
-122.15107350000001 37.74909219
-122.1510672 37.74961588
-122.15106340000001 37.76483982
-122.1510564 37.75384568
-122.15104270000002 37.75424033
-122.15099529999999 37.69979788
-122.1509503 37.74884012
-122.15094369999998 37.74936388
-122.1509003 37.76430375
-122.1508922 37.69951614
-122.1508825 37.75407811
-122.15087549999998 37.76461434
-122.1508449 37.75446033
-122.1508271 37.74858806
-122.15082029999999 37.74911189
-122.1507386 37.69962039
-122.15071

-122.11756470000002 37.67652989
-122.11752129999999 37.67655566
-122.1175132 37.67686003
-122.11743909999998 37.70568143
-122.11733059999999 37.70540333
-122.11730990000001 37.67635079
-122.1172557 37.67668332
-122.11721770000001 37.67667765
-122.1171511 37.70553727
-122.1170561 37.67617072
-122.117043 37.70525872
-122.11700149999999 37.67650364
-122.11686429999999 37.70539176
-122.1168061 37.67598737
-122.116758 37.70511104
-122.11674950000001 37.67632203
-122.11674479999999 37.70517992
-122.116579 37.70524444
-122.1165584 37.67580205
-122.116501 37.67613741
-122.11647590000001 37.70495972
-122.1163152 37.67561306
-122.116297 37.705093100000006
-122.11625459999999 37.67595096
-122.11619679999998 37.70480513
-122.11607190000001 37.67542406
-122.1160173 37.70493904
-122.1160107 37.67576252
-122.11591999999999 37.70464775
-122.11582859999999 37.67523507
-122.11576670000001 37.67557409
-122.1157398 37.70478253
-122.1156456 37.70448792
-122.1155813 37.67504945
-122.1155228 37.67538565
-122

-122.07815020000001 37.62302827
-122.07798490000002 37.62239612
-122.07798229999999 37.62279318
-122.0778176 37.62216076
-122.07781440000001 37.62255809
-122.07778290000002 37.69090976
-122.0777045 37.62200178
-122.07764990000001 37.62192554
-122.07764650000001 37.62232301
-122.07756100000002 37.62175669
-122.0774817 37.62169061
-122.0774785 37.62208792
-122.07744270000002 37.69091232
-122.07741740000002 37.62151161
-122.07731340000001 37.62145567
-122.07731059999999 37.62185283
-122.07727379999999 37.62126652
-122.07714509999998 37.62122073
-122.0771429 37.62161766
-122.07712190000001 37.62102483
-122.0771024 37.69091487
-122.0769769 37.62098579999999
-122.0769752 37.62138245
-122.07695729999999 37.62078824
-122.07680859999999 37.62075086
-122.0768076 37.62114724
-122.0767928 37.62055166
-122.07676219999999 37.69091743
-122.07664040000002 37.62051592
-122.0766399 37.62091202
-122.07663670000001 37.62031165
-122.07649509999999 37.62006585
-122.0764723 37.62067681
-122.07647209999999 37

In [56]:
daymet_average_data_all = daymet_avg_data_top + daymet_avg_data_bottom

pool = ThreadPool(processes=10)
start_year = 2015
end_year = 2016


async_result = [] #Create an empty list which stores the 
#return object of the function call. Results of the function call are later obtained using get()

for row in df_measured.tail(11488).iterrows():
    lon, lat = (row[1]['Longitude'], row[1]['Latitude'])
    thread_result = pool.apply_async(make_api_call, (lat, lon, start_year, end_year))
    async_result.append(thread_result)
    print(lon,lat)

#Note that the function name and arguments are passed differently to the apply_async function. 
#async_result now stores the returned object of the make_api_call
    
    
daymet_average_data_all = [] #This list stores the result of the api call for all the latitudes and longitudes
    
for res in async_result: #loop through each async_result object
    df_daymet_api = res.get() #using the .get() function to get the result
    if df_daymet_api is not None:
        # get the return value from function.
        daymet_api_avg_data_lat_lon = daymet_avg(df_daymet_api, start_year, end_year)  
        ## create a list with daymet parameters for each location
        daymet_average_data_all.append(daymet_api_avg_data_lat_lon) 
    else:
        daymet_average_data_all.append(None) 

In [ ]:
#Identify the rows that did not return any value from the API call
resnone = [i for i, val in enumerate(daymet_average_data_all) if val == None] 

In [ ]:
len(resnone)

**Make the API call again with just the rows that did not return any value with the API call. Append the results of this to daymet_average_data_all created above.**

pool = ThreadPool(processes=1)
start_year = 2015
end_year = 2016


async_result = [] #Create an empty list which stores the 
#return object of the function call. Results of the function call are later obtained using get()

for res in resnone:
    lon, lat = (df_measured.iloc[res]['Longitude'], df_measured.iloc[res]['Latitude'])
    thread_result = pool.apply_async(make_api_call, (lat, lon, start_year, end_year))
    async_result.append(thread_result)
    print(lon,lat)

#Note that the function name and arguments are passed differently to the apply_async function. 
#async_result now stores the returned object of the make_api_call
    
    

    
for res in async_result: #loop through each async_result object
    df_daymet_api = res.get() #using the .get() function to get the result
    if df_daymet_api is not None:
        # get the return value from function.
        daymet_api_avg_data_lat_lon = daymet_avg(df_daymet_api, start_year, end_year)  
        ## create a list with daymet parameters for each location
        daymet_average_data_all.append(daymet_api_avg_data_lat_lon) 
    else:
        daymet_average_data_all.append(None) 

**Drop the rows that have none values in the list and store it in a separate list. The combine both lists**

In [ ]:
daymet_average_data_all_2 = list(filter(None, daymet_average_data_all)) 

## <b> <font size = 4>  Convert both lists into dataframes</b> </font>

In [57]:
compute_daymet_average = pd.DataFrame(daymet_average_data_all) #convert list to a dataframe    

In [ ]:
compute_daymet_average_df_2 = pd.DataFrame(daymet_average_data_all_2) #convert list to a dataframe    

In [ ]:
compute_daymet_average_df = compute_daymet_average_df_1.append(compute_daymet_average_df_2)

In [58]:
compute_daymet_average.head()

,0,1,2,3,4,5
0,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503
1,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503
2,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503
3,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503
4,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503


In [59]:
# Add the latitude and longitude from df_measured to the daymet_average dataframe
daymet_average_all_df = compute_daymet_average.join(df_measured)

In [60]:
daymet_average_all_df.head()

,0,1,2,3,4,5,Latitude,Longitude
0,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503,37.806781,-122.322594
1,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503,37.806150,-122.322310
2,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503,37.806420,-122.322301
3,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503,37.805880,-122.322299
4,43223.606557,1.650273,341.831694,20.744536,11.528689,922.076503,37.806689,-122.322267


In [61]:
daymet_average_all_df.columns = ['Daylen','Precip','Radiation','Maxtemp','Mintemp','Pressure','Latitude','Longitude']

In [62]:
daymet_average_all_df.shape

(21488, 8)

##### Write to a csv file
daymet_average_all_df.to_csv("Data/daymet_avg_15_16.csv")

## <b> <font size = 4>  Call daymet function on gridpoints </b> </font>

In [7]:
## Set the bounding box for Oakland/Berkeley area and identify points that are not in land
bm = Basemap(llcrnrlon = -122.33, llcrnrlat =37.67, urcrnrlon = -122.10 , urcrnrlat =  37.9, area_thresh = 0.1, resolution = 'l' )   # default: projection='cyl'
for index, values in df_grid.iterrows(): 
    df_grid.loc[index, 'is_land'] = str(bm.is_land(values[1], values[0]))


/home/shridhar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  


In [8]:
df_grid_filter = df_grid[df_grid['is_land'] == 'True']

In [9]:
df_grid_filter.shape

(2411, 3)

In [28]:
pool = ThreadPool(processes=20)
start_year = 2015
end_year = 2016


async_result = [] #Create an empty list which stores the return object of the function call. 
                  #Results of the function call are later obtained using get()

    
# Run through the first 10,000 rows in the dataframe    
for row in df_grid_filter.iterrows():
    lon, lat = (row[1]['Longitude'], row[1]['Latitude'])
    thread_result = pool.apply_async(make_api_call, (lat, lon, start_year, end_year))
    async_result.append(thread_result)

#Note that the function name and arguments are passed differently to the apply_async function. 
#async_result now stores the returned object of the make_api_call

    
daymet_average_data_all = [] #This list stores the result of the api call for all the latitudes and longitudes

for res in async_result: #loop through each async_result object
    df_daymet_api = res.get() #using the .get() function to get the result
    if df_daymet_api is not None:
        # get the return value from function.
        daymet_api_avg_data_lat_lon = daymet_avg(df_daymet_api, start_year, end_year)  
        ## create a list with daymet parameters for each location
        daymet_average_data_all.append(daymet_api_avg_data_lat_lon) 
    else:
        daymet_average_data_all.append(None) 

In [29]:
daymet_grid_data =daymet_average_data_all

In [30]:
daymet_grid_data_df = pd.DataFrame(daymet_grid_data)

In [31]:
daymet_grid_data_df.head()

,0,1,2,3,4,5
0,43223.606557,1.571038,340.695082,20.613388,11.554645,919.890710
1,43223.606557,1.571038,340.695082,20.613388,11.554645,919.890710
2,43223.606557,1.571038,340.695082,20.613388,11.554645,919.890710
3,43223.606557,1.571038,340.695082,20.613388,11.554645,919.890710
4,43223.606557,1.592896,341.446994,20.665301,11.539617,923.060109


In [32]:
daymet_grid_data_df.rename(columns = {0:'dayl', 1:'prcp', 2:'srad', 3:'tmax', 4:'tmin', 5:'vp'}, inplace=True)

In [33]:
daymet_grid_data_df.drop(columns = 'dayl', inplace=True)

In [34]:
daymet_grid_data_df = daymet_grid_data_df.join(df_grid)

In [35]:
daymet_grid_data_df.head()

,prcp,srad,tmax,tmin,vp,Latitude,Longitude,is_land
0,1.571038,340.695082,20.613388,11.554645,919.890710,37.798945,-122.329059,True
1,1.571038,340.695082,20.613388,11.554645,919.890710,37.798945,-122.326624,True
2,1.571038,340.695082,20.613388,11.554645,919.890710,37.801513,-122.326624,True
3,1.571038,340.695082,20.613388,11.554645,919.890710,37.804080,-122.326624,True
4,1.592896,341.446994,20.665301,11.539617,923.060109,37.806647,-122.326624,True


In [36]:
daymet_grid_data_df.isna().sum()

prcp         26
srad         26
tmax         26
tmin         26
vp           26
Latitude      0
Longitude     0
is_land       0
dtype: int64

In [37]:
daymet_grid_data_df.to_csv("Data/daymet_grid_points_100.csv")